In [1]:
import pandas as pd
import numpy as np
import os

from env import get_db_url
from acquire import get_db_log_data

# Acquire:

I need to figure out why these are coming out differently after the join. 
- There's a difference of 52,887 records.
    - It looks like the difference here is due to having 52,893 null values in logs.cohort_id.
    - There are 6 cohorts in the cohorts_id table that do not have any server traffic, so perhaps they didn't exist.
    - 52,887 that didn't have anything to tie them to the cohorts table. Probably cohort ids - come back and verify.
- it appears that there there are some IDs and Cohort IDs that do not much.

## Importing from the database using SQL

In [2]:
df = get_db_log_data()

In [3]:
df.sample(5)

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
440099,2020-01-27,15:18:53,java-i/control-statements-and-loops,519,52.0,97.105.19.58,52.0,Europa,#europa,2019-11-04,2020-04-17,2019-11-04 18:27:07,2019-11-04 18:27:07,None,2.0
389147,2019-11-05,13:32:02,java-i/introduction-to-java,426,32.0,104.189.72.26,32.0,Betelgeuse,#betelgeuse,2019-05-28,2019-10-08,2019-05-28 18:41:05,2019-05-28 18:41:05,None,2.0
107138,2018-08-17,07:00:11,html-css/css-i/box-model,218,25.0,70.94.153.55,25.0,Wrangell,#wrangell,2018-07-23,2018-11-29,2018-07-23 15:02:25,2018-07-23 15:02:25,None,2.0
824288,2021-02-22,10:18:59,classification/acquire,847,133.0,173.174.194.60,133.0,Easley,#easley,2020-12-07,2021-06-08,2020-12-07 15:20:18,2020-12-07 15:20:18,None,3.0
511414,2020-04-14,13:24:08,toc,612,56.0,172.14.176.83,56.0,Apex,#apex,2020-02-24,2020-07-29,2020-02-24 15:39:33,2020-07-29 18:41:13,None,2.0
